In [29]:
#Maria Izzi
#July 2021

In [45]:
#importing necessary packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd
import PySimpleGUI as sg
import re
import requests

In [46]:
#setting up GUI
sg.theme('Black')
layout = [[sg.Text('Instructions: tinyurl.com/nvctscraper')],
          [sg.Text('Input file:'), sg.InputText()],
          [sg.Text('Parcel ID column name:'), sg.InputText()],
          [sg.Text('County:')],
          [sg.Listbox(['Arlington','Fairfax','Loudoun','Stafford'],size=(20,4))],
          [sg.Text('County parcel ID search website link:'), sg.InputText()],
          [sg.Button('Done')]]

window = sg.Window('NVCT Parcel Web Scraper', layout)

while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'Cancel' or event=='Done': # if user closes window or clicks cancel
        break
    print('You entered ', values)

window.close()

In [47]:
#taking inputs from GUI
inp_file = values[0]
col_name = values[1]
county = values[2]
link = values[3]

In [48]:
#importing data
data = pd.read_csv(inp_file,dtype=object)

In [49]:
#ARLINGTON (Stafford is similar)
#setting up list for invalid parcel IDs
invalids = []
if(county[0]=='Arlington'):
    #setting up webdriver
    driver = webdriver.Edge('./msedgedriver')
    
    #setting up output dataframe
    df = pd.DataFrame(data={'PIN':[],'Name':[],'Address':[]})
    
    #count for adding next line to output dataframe
    count = 0
    
    #iterating through each parcel ID in the given column in the data
    for pin in data[col_name]:
        try:
            #open the website
            driver.get(link)
            
            #search for the search box and enter the parcel ID
            temp = driver.find_element_by_id("SearchFilters_RPCs")
            temp.send_keys(pin)
            temp.submit()
            
            #open a BeautifulSoup parser for the website
            src = driver.page_source
            soup = bs(src, 'html.parser')
            
            #open the link for the page with the parcel owner(s) name and address
            allread = soup.find_all('a',string='View')

            viewlink = ""
            for i in allread:
                viewlink = i.get('href')

            r = requests.get("https://propertysearch.arlingtonva.us/"+viewlink)
            soup_arlington = bs(r.content,'html.parser')
            arl_read = [soup_arlington.find_all(class_="one-half first")[0].text,soup_arlington.find_all(class_="one-half first")[1].text]

            df.loc[count] = ([pin,arl_read[0].replace("Owner","",1).strip(),arl_read[1].replace("Mailing Address","",1).strip()])
            count += 1
            
        #to make sure any invalid parcel IDs don't halt the whole process
        except:
            print(pin+" is invalid.")
            invalids.append(pin)
    driver.quit()

In [50]:
# FAIRFAX (Loudoun is similar)
invalids = []
if(county[0]=='Fairfax'):
    driver = webdriver.Edge('./msedgedriver')

    df = pd.DataFrame(data={'PIN':[],'Name':[],'Address':[]})
    count = 0

    for pin in data[col_name]:
        try:
            driver.get(link)
            temp = driver.find_element_by_id("inpParid")
            temp.send_keys(pin)
            temp.send_keys(Keys.ENTER)
            src = driver.page_source
            soup = bs(src, 'html.parser')
            
            #no separate page to open, all data available on this page, so just selecting the data directly
            allread = soup.find_all('td',"DataletData")
            df.loc[count] = ([pin,allread[0].text,allread[1].text])
            count += 1
        except:
            print(pin+" is invalid.")
            invalids.append(pin)

    driver.quit()

In [51]:
#LOUDOUN (Fairfax is similar)
if(county[0]=='Loudoun'):
    driver = webdriver.Edge('./msedgedriver')

    df = pd.DataFrame(data={'PIN':[],'Name':[],'Care Of':[],'Address':[]})
    count = 0

    for pin in data[col_name]:
        try:
            driver.get(link)
            temp = driver.find_element_by_id("inpParid")
            temp.send_keys(pin)
            temp.send_keys(Keys.ENTER)
            src = driver.page_source
            soup = bs(src, 'html.parser')
            allread = soup.find_all('td',"DataletData")
            df.loc[count] = ([pin,allread[0].text,allread[1].text,allread[2].text+"\n"+allread[4].text])
            count += 1    
        except:
            print(pin+" is invalid.")
            invalids.append(pin)
    driver.quit()

In [63]:
#STAFFORD (Arlington is similar)
invalids = []
if(county[0]=='Stafford'):
    driver = webdriver.Edge('./msedgedriver')
    df = pd.DataFrame(data={'PIN':[],'Name':[],'Address':[]})

    count = 0

    for pin in data[col_name]:
        try:
            driver.get(link)
            temp = driver.find_element_by_id("fldSearchFor")
            temp.send_keys(pin)
            temp.submit()
            src = driver.page_source
            soup = bs(src, 'html.parser')
            allread = soup.find_all('a',title="Click here to view information")

            viewlink = ""
            for i in allread:
                viewlink = i.get('href')

            r = requests.get("http://va-stafford-assessor.publicaccessnow.com"+viewlink)
            soup_stafford = bs(r.content,'html.parser')
            staff_read = soup_stafford.find_all(class_="ui-widget ui-widget-content center")[0].text

            staff_read_list = staff_read.strip().split("\n")
            staff_read_list[0] = staff_read_list[0].strip()
            staff_read_list[1] = staff_read_list[1].strip()+"\n"+staff_read_list[2].strip()
            
            df.loc[count] = ([pin,staff_read_list[0],staff_read_list[1]])
            count += 1
        except:
            print(pin+" is invalid.")
            invalids.append(pin)
    driver.quit()

In [64]:
df.to_csv(r'webscraper_output.csv',index=False)

In [39]:
sg.theme('Black')   # Add a touch of color
# All the stuff inside your window.
layout = [[sg.Text('Done! The output file should be in the same location as the input file.')],
          [sg.Text('These parcel IDs (if any) are invalid:')],
          [sg.Text(invalids)]]

# Create the Window
window = sg.Window('NVCT Parcel Web Scraper', layout)
# Event Loop to process "events" and get the "values" of the inputs
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED: # if user closes window or clicks cancel
        break

window.close()